In [ ]:
# Enable auto-reloading of modules
%load_ext autoreload
%autoreload 2

In [1]:
from nhl_scraper import scraper
import polars as pl
from tqdm import tqdm

!playwright install

from nhl_scraper.scraper import NHLGameURLGenerator, NHLPlayByPlayParser, NHLDataScraper, NHLBaseScraper

scraper = scraper.Scraper()


Welcome to the NHL Scraper!

My name is Max and I'm the developer of this project and hockey fan. I hope you enjoy using this tool!

This is a work in progress and not all functionalities are available yet.

*** IMPORTANT ***

Make sure to run the following command in the terminal (it might take a while, but it will ensure that the playwright browser is installed):

$ playwright install
or
$ playwright install chromium

***

If you have any questions, please contact me on X: @woumaxx[https://x.com/woumaxx]

Thank you for using the NHL Scraper!





In [19]:
pbp = pl.read_parquet("/Users/max/Documents/nhl_scraper/data/pbp.parquet")
pbp.head()

#,Description,Event,Per,Str,api_event,awayGId,awayGName,awaySOG,awayScore,awaySktrs,awayTeam,awayTeamId,away_skater_fullName1,away_skater_fullName2,away_skater_fullName3,away_skater_fullName4,away_skater_fullName5,away_skater_fullName6,away_skater_fullName7,away_skater_id1,away_skater_id2,away_skater_id3,away_skater_id4,away_skater_id5,away_skater_id6,away_skater_id7,details_assist1PlayerTotal,details_assist2PlayerTotal,details_descKey,details_drawnByPlayerId,details_duration,details_playerId,details_scoringPlayerTotal,details_secondaryReason,details_servedByPlayerId,details_typeCode,…,home_skater_id16,home_skater_id17,home_skater_id2,home_skater_id3,home_skater_id4,home_skater_id5,home_skater_id6,home_skater_id7,home_skater_id8,home_skater_id9,index,limitedScoring,meta_datetime,periodNumber,periodType,playerId_1,playerId_2,playerId_3,playerName_1,playerName_2,playerName_3,reason,season,shotType,startTimeUTC,strength,timeInPeriod,timeInPeriod_s,timeRemaining,venue,venueLocation,venueUTCOffset,x,xCoord,y,yCoord,zoneCode
str,str,str,str,str,str,i64,str,i64,i64,u32,str,str,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,str,i64,str,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,bool,datetime[μs],i64,str,i64,i64,i64,str,str,str,str,i32,str,datetime[μs],str,str,i64,str,str,str,str,f64,f64,f64,f64,str
"""4""","""Period Start- Local time: 7:14…","""PSTR""","""1""","""""","""period-start""",8474593,"""Jacob Markstrom""",0,0,5,"""NJD""","""1""","""Nico Hischier""","""Ondrej Palat""","""Tomas Tatar""","""Simon Nemec""","""Jonas Siegenthaler""",null,null,8480002,8476292,8475193,8483495,8478399,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,8477979,8479359,8480035,8480839,null,null,null,null,0,false,2025-03-21 08:14:08.670442,1,"""REG""",null,null,null,null,null,null,null,20242025,null,2024-10-04 17:00:00,null,"""00:00""",0,"""20:00""","""O2 Czech Republic""","""Prague""","""+02:00""",null,null,null,null,null
"""5""","""NJD won Neu. Zone - NJD #13 HI…","""FAC""","""1""","""EV""","""faceoff""",8474593,"""Jacob Markstrom""",0,0,5,"""NJD""","""1""","""Nico Hischier""","""Ondrej Palat""","""Tomas Tatar""","""Simon Nemec""","""Jonas Siegenthaler""",null,null,8480002,8476292,8475193,8483495,8478399,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,8477979,8479359,8480035,8480839,null,null,null,null,1,false,2025-03-21 08:14:08.670442,1,"""REG""",8480002,8478043,null,"""Nico Hischier""","""Sam Lafferty""",null,null,20242025,null,2024-10-04 17:00:00,"""5v5""","""00:00""",0,"""20:00""","""O2 Czech Republic""","""Prague""","""+02:00""",-0.0,0.0,-0.0,0.0,"""N"""
"""6""","""BUF #29 MALENSTYN HIT NJD #90 …","""HIT""","""1""","""EV""","""hit""",8474593,"""Jacob Markstrom""",0,0,5,"""NJD""","""1""","""Nico Hischier""","""Ondrej Palat""","""Tomas Tatar""","""Simon Nemec""","""Jonas Siegenthaler""",null,null,8480002,8476292,8475193,8483495,8478399,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,8477979,8479359,8480035,8480839,null,null,null,null,2,false,2025-03-21 08:14:08.670442,1,"""REG""",8479359,8475193,null,"""Beck Malenstyn""","""Tomas Tatar""",null,null,20242025,null,2024-10-04 17:00:00,"""5v5""","""00:05""",5,"""19:55""","""O2 Czech Republic""","""Prague""","""+02:00""",-69.0,69.0,40.0,-40.0,"""N"""
"""7""","""NJD ONGOAL - #17 NEMEC, Wrist …","""SHOT""","""1""","""EV""","""shot-on-goal""",8474593,"""Jacob Markstrom""",1,0,5,"""NJD""","""1""","""Nico Hischier""","""Ondrej Palat""","""Tomas Tatar""","""Simon Nemec""","""Jonas Siegenthaler""",null,null,8480002,8476292,8475193,8483495,8478399,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,8477979,8479359,8480035,8480839,null,null,null,null,3,false,2025-03-21 08:14:08.670442,1,"""REG""",8483495,null,null,"""Simon Nemec""",null,null,null,20242025,"""wrist""",2024-10-04 17:00:00,"""5v5""","""00:08""",8,"""19:52""","""O2 Czech Republic""","""Prague""","""+02:00""",-57.0,57.0,40

In [20]:
event_mapping = {
            "BLOCK": "blocked-shot",
            "DELPEN": "delayed-penalty",
            "FAC": "faceoff",
            "GIVE": "giveaway",
            "GOAL": "goal",
            "HIT": "hit",
            "MISS": "missed-shot",
            "PENL": "penalty",
            "SHOT": "shot-on-goal",
            "STOP": "stoppage",
            "TAKE": "takeaway",
            "GEND": "game-end",
            "PEND": "period-end",
            "PSTR": "period-start",
            "SOC": "shootout-completed",
        }
events_in_code = set(event_mapping.keys())
events_html = set(pbp["Event"].unique())
events_not_in_code = events_html - events_in_code
events_not_in_code


{'CHL', 'EIEND', 'EISTR', 'GOFF', 'SPC'}

In [36]:
# Reorder columns
pbp.filter(pl.col("Event").is_in(['CHL']))#.select("Description", "Event", 'eventOwnerTeam', "Str", "gameId")

#,Description,Event,Per,Str,api_event,awayGId,awayGName,awaySOG,awayScore,awaySktrs,awayTeam,awayTeamId,away_skater_fullName1,away_skater_fullName2,away_skater_fullName3,away_skater_fullName4,away_skater_fullName5,away_skater_fullName6,away_skater_fullName7,away_skater_id1,away_skater_id2,away_skater_id3,away_skater_id4,away_skater_id5,away_skater_id6,away_skater_id7,details_assist1PlayerTotal,details_assist2PlayerTotal,details_descKey,details_drawnByPlayerId,details_duration,details_playerId,details_scoringPlayerTotal,details_secondaryReason,details_servedByPlayerId,details_typeCode,…,home_skater_id16,home_skater_id17,home_skater_id2,home_skater_id3,home_skater_id4,home_skater_id5,home_skater_id6,home_skater_id7,home_skater_id8,home_skater_id9,index,limitedScoring,meta_datetime,periodNumber,periodType,playerId_1,playerId_2,playerId_3,playerName_1,playerName_2,playerName_3,reason,season,shotType,startTimeUTC,strength,timeInPeriod,timeInPeriod_s,timeRemaining,venue,venueLocation,venueUTCOffset,x,xCoord,y,yCoord,zoneCode
str,str,str,str,str,str,i64,str,i64,i64,u32,str,str,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,str,i64,str,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,bool,datetime[μs],i64,str,i64,i64,i64,str,str,str,str,i32,str,datetime[μs],str,str,i64,str,str,str,str,f64,f64,f64,f64,str
"""60""","""PIT Challenge - Offside - Res…","""CHL""","""1""","""EV""",null,8478048,"""Igor Shesterkin""",3,1,5,"""NYR""","""3""","""Filip Chytil""","""Kaapo Kakko""","""Will Cuylle""","""Zac Jones""","""Victor Mancini""",null,null,8480078,8481554,8482157,8481708,8483768,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,8483487,8479344,8481030,8477435,null,null,null,null,56,false,2025-03-21 08:14:16.199183,null,null,null,null,null,null,null,null,null,20242025,null,2024-10-09 23:30:00,null,null,null,null,"""PPG Paints Arena""","""Pittsburgh""","""-04:00""",null,null,null,null,null
"""24""","""MTL Challenge - Interference o…","""CHL""","""1""","""SH""",null,8480051,"""Cayden Primeau""",0,0,4,"""MTL""","""8""","""Christian Dvorak""","""Josh Anderson""","""Mike Matheson""","""David Savard""",null,null,null,8477989,8476981,8476875,8475233,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,8475745,8479987,8479638,8476854,null,null,null,null,20,false,2025-03-21 08:14:20.221293,null,null,null,null,null,null,null,null,null,20242025,null,2024-10-10 23:00:00,null,null,null,null,"""TD Garden""","""Boston""","""-04:00""",null,null,null,null,null
"""199""","""DET Challenge - Missed stoppag…","""CHL""","""2""","""EV""",null,8482446,"""Joel Blomqvist""",13,4,5,"""PIT""","""5""","""Evgeni Malkin""","""Michael Bunting""","""Drew O'Connor""","""Marcus Pettersson""","""Kris Letang""",null,null,8471215,8478047,8482055,8477969,8471724,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,8474141,8482078,8473507,8481542,null,null,null,null,195,false,2025-03-21 08:14:23.478363,null,null,null,null,null,null,null,null,null,20242025,null,2024-10-11 00:00:00,null,null,null,null,"""Little Caesars Arena""","""Detroit""","""-04:00""",null,null,null,null,null
"""85""","""UTA Challenge - Interference o…","""CHL""","""1""","""EV""",null,8478971,"""Connor Ingram""",3,0,5,"""UTA""","""59""","""Alexander Kerfoot""","""Kevin Stenlund""","""Logan Cooley""","""Michael Kesselring""","""Ian Cole""",null,null,8477021,8478831,8483431,8480891,8474013,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,8477500,8484958,8477407,8480865,null,null,null,null,81,false,2025-03-21 08:14:26.495690,null,null,null,null,null,null,null,null,null,20242025,null,2024-10-10 23:30:00,null,null,null,null,"""UBS Arena""","""Elmont""","""-04:00""",null,null,null,null,null
"""150""","""MIN Challenge - Interference o…","""CHL""","""2""","""SH""",null,8478007,"""Elvis Merzlikins""",16,1,5,"""CBJ""","""29""","""Adam Fantilli""","""Kevin Labanc""","""Kirill Marchenko""","""James van Riem

In [34]:
pbp.columns


['#',
 'Description',
 'Event',
 'Per',
 'Str',
 'api_event',
 'awayGId',
 'awayGName',
 'awaySOG',
 'awayScore',
 'awaySktrs',
 'awayTeam',
 'awayTeamId',
 'away_skater_fullName1',
 'away_skater_fullName2',
 'away_skater_fullName3',
 'away_skater_fullName4',
 'away_skater_fullName5',
 'away_skater_fullName6',
 'away_skater_fullName7',
 'away_skater_id1',
 'away_skater_id2',
 'away_skater_id3',
 'away_skater_id4',
 'away_skater_id5',
 'away_skater_id6',
 'away_skater_id7',
 'details_assist1PlayerTotal',
 'details_assist2PlayerTotal',
 'details_descKey',
 'details_drawnByPlayerId',
 'details_duration',
 'details_playerId',
 'details_scoringPlayerTotal',
 'details_secondaryReason',
 'details_servedByPlayerId',
 'details_typeCode',
 'easternUTCOffset',
 'elapsedTime',
 'event',
 'eventOwnerTeam',
 'eventOwnerTeamId',
 'gameDate',
 'gameId',
 'gameOutcome',
 'gameScheduleState',
 'gameState',
 'gameType',
 'goalieInNetId',
 'goalieInNetName',
 'homeGId',
 'homeGName',
 'homeSOG',
 'homeSco

In [31]:
game = await scraper.scrape_pbp(game_ids=[2024020199])

game.filter(pl.col("Per") == "1").tail(10)

Scraping NHL PBP Data: 100%|██████████| 1/1 [00:01<00:00,  1.75s/it]


#,Per,Str,Event,Description,homeGName,awayGName,homeGId,awayGId,index,event,timeInPeriod,timeRemaining,homeTeamDefendingSide,api_event,periodNumber,periodType,eventOwnerTeamId,xCoord,yCoord,zoneCode,reason,shotType,goalieInNetId,details_playerId,details_typeCode,details_descKey,details_duration,details_drawnByPlayerId,awaySOG,homeSOG,details_secondaryReason,details_scoringPlayerTotal,details_assist1PlayerTotal,details_assist2PlayerTotal,awayScore,homeScore,…,home_skater_fullName2,home_skater_fullName3,home_skater_fullName4,home_skater_fullName5,home_skater_fullName6,away_skater_fullName1,away_skater_fullName2,away_skater_fullName3,away_skater_fullName4,away_skater_fullName5,away_skater_fullName6,homeSktrs,awaySktrs,eventOwnerTeam,awayTeam,homeTeam,awayTeamId,homeTeamId,season,gameType,limitedScoring,gameDate,venue,venueLocation,startTimeUTC,easternUTCOffset,venueUTCOffset,gameState,gameScheduleState,gameOutcome,x,y,timeInPeriod_s,elapsedTime,strength,gameId,meta_datetime
str,str,str,str,str,str,str,i64,i64,i64,str,str,str,str,str,i64,str,str,f64,f64,str,str,str,i64,i64,str,str,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,…,str,str,str,str,null,str,str,str,str,str,str,u32,u32,str,str,str,str,str,i32,i32,bool,datetime[μs],str,str,datetime[μs],str,str,str,str,str,f64,f64,i64,i64,str,str,datetime[μs]
"""80""","""1""","""EV""","""BLOCK""","""STL #76 BOLDUC OPPONENT-BLOCKE…","""Jordan Binnington""","""Andrei Vasilevskiy""",8476412,8476883,76,"""blocked-shot""","""18:25""","""01:35""","""left""","""blocked-shot""",1,"""REG""","""19""",43.0,-31.0,"""D""","""blocked""",null,null,null,null,null,null,null,9,3,null,null,null,null,0,0,…,"""Zack Bolduc""","""Alexey Toropchenko""","""Justin Faulk""","""P.O Joseph""",null,"""Luke Glendening""","""Michael Eyssimont""","""Zemgus Girgensons""","""Ryan McDonagh""","""Nick Perbix""",null,5,5,"""STL""","""TBL""","""STL""","""14""","""19""",20242025,2,false,2024-11-05 00:00:00,"""Enterprise Center""","""St. Louis""",2024-11-06 01:00:00,"""-05:00""","""-06:00""","""OFF""","""OK""","""REG""",43.0,-31.0,1105,1105,"""5v5""","""2024020199""",2025-03-24 13:01:11.713040
"""81""","""1""","""EV""","""MISS""","""TBL #14 GEEKIE, Wrist, Wide Le…","""Jordan Binnington""","""Andrei Vasilevskiy""",8476412,8476883,77,"""missed-shot""","""18:44""","""01:16""","""left""","""missed-shot""",1,"""REG""","""14""",-77.0,-9.0,"""O""","""wide-left""","""wrist""",8476412,null,null,null,null,null,9,3,null,null,null,null,0,0,…,"""Brandon Saad""","""Pavel Buchnevich""","""Justin Faulk""","""P.O Joseph""",null,"""Conor Geekie""","""Michael Eyssimont""","""Gage Goncalves""","""Ryan McDonagh""","""Erik Cernak""",null,5,5,"""TBL""","""TBL""","""STL""","""14""","""19""",20242025,2,false,2024-11-05 00:00:00,"""Enterprise Center""","""St. Louis""",2024-11-06 01:00:00,"""-05:00""","""-06:00""","""OFF""","""OK""","""REG""",-77.0,-9.0,1124,1124,"""5v5""","""2024020199""",2025-03-24 13:01:11.713040
"""82""","""1""","""""","""STOP""","""ON-ICE OFFICIAL,VIDEO REVIEW""","""Jordan Binnington""","""Andrei Vasilevskiy""",8476412,8476883,78,"""stoppage""","""18:49""","""01:11""","""left""","""stoppage""",1,"""REG""",null,null,null,null,"""referee-or-linesman""",null,null,null,null,null,null,null,9,3,"""video-review""",null,null,null,0,0,…,"""Brandon Saad""","""Pavel Buchnevich""","""Justin Faulk""","""P.O Joseph""",null,"""Conor Geekie""","""Michael Eyssimont""","""Gage Goncalves""","""Ryan McDonagh""","""Erik Cernak""",null,5,5,null,"""TBL""","""STL""","""14""","""19""",20242025,2,false,2024-11-05 00:00:00,"""Enterprise Center""","""St. Louis""",2024-11-06 01:00:00,"""-05:00""","""-06:00""","""OFF""","""OK""","""REG""",null,null,1129,1129,null,"""2024020199""",2025-03-24 13:01:11.713040
"""83""","""1""","""""","""STOP""","""INJURY - PLAYER""","""Jordan Binnington""","""Andrei Vasilevskiy""",8476412,8476883,79,"""stoppage""","""18:49""","""01:11""","""left""","""stoppage""",1,"""REG""",null,null,null,null,"""player-injury""",null,null,null,null,null,